In [ ]:
from langgraph.graph import StateGraph, END, START
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from typing import TypedDict, Annotated


In [5]:
load_dotenv()

# LLM 
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")